## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001FA4B29EA20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001FA4B6DBE90>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is achraf and I am a ML Engineer and also reserach in IA and ML")])

AIMessage(content="Hello Achraf! It's great to meet you. \n\nAs a large language model, I'm always interested in learning more about the work of other AI and ML professionals. What kind of research are you currently involved in? \n\nI'm eager to hear about your projects and any insights you have on the future of AI and ML.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 30, 'total_tokens': 105, 'completion_time': 0.136363636, 'prompt_time': 0.001424299, 'queue_time': 0.186326551, 'total_time': 0.137787935}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--abd14fce-f056-478f-bc1f-6a63cd880c5d-0', usage_metadata={'input_tokens': 30, 'output_tokens': 75, 'total_tokens': 105})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is achraf and I am a ML Engineer and also reserach in IA and ML"),
        AIMessage(content="HHello Krish, it's nice to meet you! \n\nThat's an impressive title. I'm curious, what kind of AI projects are you working on? Perhaps you could tell me about some of the challenges and rewards of being a Chief AI Engineer?  \n\nI'm always eager to learn more about how AI is being used in the world.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Achraf, and you are a Machine Learning Engineer and researcher in AI and ML. \n\nIs there anything else you'd like me to remember about you? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 128, 'total_tokens': 168, 'completion_time': 0.072727273, 'prompt_time': 0.003520079, 'queue_time': 0.186108891, 'total_time': 0.076247352}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--961140d2-163c-4fa6-a25c-3440388b0c15-0', usage_metadata={'input_tokens': 128, 'output_tokens': 40, 'total_tokens': 168})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]



In [7]:

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [8]:
config={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is achraf and I am a AI Engineer")],
    config=config
)

In [9]:
response.content

"Hi Achraf, it's nice to meet you!\n\nAs an AI, I don't have personal experiences or feelings, but I'm always eager to learn about what people are doing in the field of AI. \n\nWhat kind of work do you do as an AI Engineer?  Are you working on any interesting projects right now?\n"

In [10]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Achraf. 😊  \n\nI remembered from our earlier conversation!  Is there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 110, 'total_tokens': 140, 'completion_time': 0.054545455, 'prompt_time': 0.002830649, 'queue_time': 0.188937785, 'total_time': 0.057376104}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--0c587d52-235a-4b56-a3af-6e74976c4306-0', usage_metadata={'input_tokens': 110, 'output_tokens': 30, 'total_tokens': 140})

`he need have same session id when you change it he doesn't hav ememoory in new session id `

In [11]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to remember it for our current interaction. 😊  \n\n"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John, it's nice to meet you! What can I do for you today? 😊 \n"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John, remember?  😄  \n\nIs there anything else I can help you with?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [14]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [15]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is achraf")]})

AIMessage(content="Hi Achraf! It's nice to meet you.  \n\nI'm happy to help with any questions you have. Just ask away! 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 32, 'total_tokens': 67, 'completion_time': 0.063636364, 'prompt_time': 0.00155798, 'queue_time': 0.187416721, 'total_time': 0.065194344}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2e4bc560-a514-402f-8f7a-5aec7690a5d0-0', usage_metadata={'input_tokens': 32, 'output_tokens': 35, 'total_tokens': 67})

In [16]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [17]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is achraf")],
    config=config
)

response

AIMessage(content="Hello Achraf! \n\nIt's nice to meet you. What can I do for you today?\n\nI'm ready for your questions. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 32, 'total_tokens': 69, 'completion_time': 0.067272727, 'prompt_time': 0.00150932, 'queue_time': 0.188622016, 'total_time': 0.068782047}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--d7e1d5e1-00d7-4b6f-aed3-bbbbcba9ae04-0', usage_metadata={'input_tokens': 32, 'output_tokens': 37, 'total_tokens': 69})

In [18]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Achraf! 😄  \n\nI remember that you told me. \n\n\n\n'

In [19]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [20]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is achraf")],"language":"french"})
response.content

"Bonjour Achraf ! \n\nEnchanté de te rencontrer. Comment puis-je t'aider aujourd'hui ? 😊 \n"

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [21]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [22]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am achraf")],"language":"arabic"},
    config=config
)
repsonse.content

'مرحبا أَشْرَاف، كيف يمكنني مساعدتك؟ 😊 \n'

In [23]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "arabic"},
    config=config,
)

In [24]:
response.content

'اسْمُكَ أَشْرَافَ. 😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [25]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

f:\drive\LLM_LCEL_application\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [26]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your ice cream preferences.  What's your favorite flavor? 😊  \n"

In [27]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2" 😊  \n\n\n\nLet me know if you have any other questions!\n'

In [28]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [30]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="who I like?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a language model, I don't have personal opinions or feelings, so I don't know who you like!  \n\nIs there someone you'd like to tell me about? 😊  \n\n"

In [31]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is.  \n\n"